In [1]:
import mediapipe as mp
import numpy as np
import torch
#import torch_geometric
from PIL import Image

from torch_geometric.data import Data
import cv2
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
import glob
import os
import math 
from IPython.display import clear_output
import itertools
import seaborn as sb
from tqdm import tqdm
import random

In [2]:

import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
kernel_size = 20



model = DenseNet121(input_shape=(kernel_size*2,kernel_size*2,3),weights=None, include_top=False)

# Choose the desired low-level feature layers
low_level_layers = [
    model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
    model.get_layer('conv3_block12_concat')# Output of the last layer in the second dense block
]
# low_level_layers = [
#     model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
#     model.get_layer('conv3_block12_concat'), 
#     model.get_layer('conv4_block24_concat')# Output of the last layer in the second dense block
# ]

# Create a new model with the desired low-level feature layers as outputs
feature_extractor = tf.keras.Model(inputs=model.input, outputs=[layer.output for layer in low_level_layers])
feature_extractor.load_weights("/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.h5")
feature_extractor.trainable=False

2024-09-19 19:30:09.707682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-19 19:30:09.710223: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-19 19:30:09.710773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-09-19 19:30:09.711429: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:

import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
kernel_size = 20



model = DenseNet121(input_shape=(kernel_size*2,kernel_size*2,3),weights=None, include_top=False)

# Choose the desired low-level feature layers
low_level_layers = [
    model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
    model.get_layer('conv3_block12_concat')# Output of the last layer in the second dense block
]
# low_level_layers = [
#     model.get_layer('conv2_block2_concat'),  # Output of the last layer in the first dense block
#     model.get_layer('conv3_block12_concat'), 
#     model.get_layer('conv4_block24_concat')# Output of the last layer in the second dense block
# ]

# Create a new model with the desired low-level feature layers as outputs
feature_extractor = tf.keras.Model(inputs=model.input, outputs=[layer.output for layer in low_level_layers])
feature_extractor.load_weights("/home/hassan-hossein/graph_landmark_face_anti_spoofing/dense_net_model.h5")
feature_extractor.trainable=False

In [4]:

#input_image = tf.random.uniform((10,40,40,3))
def DenseNet_feature_extractor(input_image) :
    #input_image = tf.keras.preprocessing.image.img_to_array(input_image)
    #print("input shape", input_image.shape)
    input_image = tf.cast(input_image, tf.float32)
    input_image = tf.keras.applications.densenet.preprocess_input(input_image)
    #input_image = tf.expand_dims(input_image, axis=0)

    feature_extracted = feature_extractor.predict(input_image,verbose=False)
    #print(feature_extracted[0].shape,feature_extracted[1].shape)
    global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[0])
    flatten = tf.keras.layers.Flatten()(global_pooling)
    total_feature =flatten
    for layer_cnt in range(1,2) : 
        global_pooling = tf.keras.layers.GlobalAveragePooling2D()(feature_extracted[layer_cnt])
        flatten = tf.keras.layers.Flatten()(global_pooling)
        total_feature = tf.concat([flatten,total_feature],1)
    return total_feature

In [5]:
kernel_size= 20
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh

connections = mp_face_mesh.FACEMESH_TESSELATION
edges = []
for connection in connections:
    edge = [connection[0], connection[1]]
    edges.append(edge)
    edge  =[connection[1], connection[0]]
    edges.append(edge)
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()

In [6]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/casia-ds/preposess/*/cropped*.jpg")

In [7]:
def random_float(f_min, f_max):
    return f_min + (f_max-f_min) * random.random()

class Contrast_and_Brightness(object):

    def __call__(self, image_x):
        gamma = random.randint(-40, 40)
        alpha = random_float(0.5, 1.5)
        image_x = cv2.addWeighted(image_x, alpha, image_x, 0, gamma)
        
        return image_x


In [8]:
arg_contranst_bright = Contrast_and_Brightness()

In [9]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = arg_contranst_bright(img)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

             patches.append(p_img)  

        patches = np.array(patches,dtype =np.float32)
        features = DenseNet_feature_extractor(patches) 
        #print(patches.shape)
        #print(patches.dtype)
        vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        torch.save(data, os.path.dirname(path)+'/arg_contrast_bright_'+os.path.basename(path)+'.pt')


Processing images:   0%|                              | 0/22296 [00:00<?, ?it/s]2024-09-19 19:30:21.039717: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2024-09-19 19:30:21.562308: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-19 19:30:21.562792: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-19 19:30:21.562806: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-09-19 19:30:21.563429: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-19 19:30:21.563474: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be o

In [10]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/oulu/preposess/*/cropped*.jpg")

In [11]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = arg_contranst_bright(img)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

             patches.append(p_img)  

        patches = np.array(patches,dtype =np.float32)
        features = DenseNet_feature_extractor(patches) 
        #print(patches.shape)
        #print(patches.dtype)
        vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        torch.save(data, os.path.dirname(path)+'/arg_contrast_bright_'+os.path.basename(path)+'.pt')


Processing images: 100%|██████████████| 136334/136334 [4:57:31<00:00,  7.64it/s]


In [12]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/Replay_attack_dataset/preposess/*/cropped*.jpg")

In [13]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = arg_contranst_bright(img)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

             patches.append(p_img)  

        patches = np.array(patches,dtype =np.float32)
        features = DenseNet_feature_extractor(patches) 
        #print(patches.shape)
        #print(patches.dtype)
        vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        torch.save(data, os.path.dirname(path)+'/arg_contrast_bright_'+os.path.basename(path)+'.pt')


Processing images: 100%|████████████████| 61989/61989 [2:14:57<00:00,  7.65it/s]


In [14]:
paths = glob.glob("/media/hassan-hossein/B660FEE360FEA8EF/MSU_MSFD_ds/MSU_MFSD/preposess/*/cropped*.jpg")

In [15]:
for path in tqdm(paths, desc="Processing images"):        
        NAME_LIST = path.split("/")[-2].split("_")
        
        #print(face_path,NAME_LIST)
        
        if "live" in NAME_LIST :
            label = 1
        elif "real" in NAME_LIST :
            label = 1
        elif "spoof" in NAME_LIST :
            label = 0
        elif "attack" in NAME_LIST :
            label = 0        
        else :
            raise 0
            print(NAME_LIST)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        img = arg_contranst_bright(img)

        #print(img.shape)

        landmark_path = os.path.dirname(path) + "/"+os.path.basename(path).split(".")[0]+".npy"

        list_of_points_face = np.load(landmark_path,allow_pickle=True).item()['landmark']
        #print(list_of_points_face)

        list_of_points_face = np.clip(list_of_points_face,0,256)

        list_of_points_face = list_of_points_face + np.array([kernel_size,kernel_size])

        img = np.pad(img, ((kernel_size, kernel_size), (kernel_size, kernel_size), (0, 0)),
                 mode='constant', constant_values=0) 


        patches= []

        list_of_points_face= np.round(list_of_points_face).astype(int)

        for (x1, y1) in list_of_points_face:

             p_img = img[y1-kernel_size:y1+kernel_size ,x1-kernel_size:x1+kernel_size]

             patches.append(p_img)  

        patches = np.array(patches,dtype =np.float32)
        features = DenseNet_feature_extractor(patches) 
        #print(patches.shape)
        #print(patches.dtype)
        vertices = torch.tensor(np.array(features), dtype=torch.float).contiguous()   

        data = Data(x=vertices, edge_index=edges,y =label)
        torch.save(data, os.path.dirname(path)+'/arg_contrast_bright_'+os.path.basename(path)+'.pt')


Processing images: 100%|██████████████████| 15549/15549 [34:03<00:00,  7.61it/s]
